# More transformations in Pandas

In this lecture we will learn about a few critical transformations that are frequently used. Those are:

- Joins
- Pivot Table
- Group by

Part of the magic of using pandas for doing transformations is that aggregation functions for **Group By** can be custom. We will solve a problem which requires us to do that.

In addition to that, we will learn a bit about MultiIndex - which is something we have to know as it's automatically created when we do a **Pivot table** or a **Group By** transformation.

In [3]:
import pandas as pd

In [4]:
codes_df = pd.read_csv('airport-codes.csv')
del codes_df['continent']
del codes_df['iata_code']
codes_df = codes_df[codes_df['type'] != 'closed']
codes_df.head(5)

,ident,type,name,elevation_ft,iso_country,iso_region,municipality,gps_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,US,US-PA,Bensalem,00A,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,US,US-KS,Leoti,00AA,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,US,US-AK,Anchor Point,00AK,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,US,US-AL,Harvest,00AL,00AL,"-86.77030181884766, 34.86479949951172"
5,00AS,small_airport,Fulton Airport,1100.0,US,US-OK,Alex,00AS,00AS,"-97.8180194, 34.9428028"


In [7]:
stats_df = pd.read_csv('us_air_transport_stats.csv')
stats_df.head(5)

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,YEAR,MONTH
0,0,0,0,156,0MQ,21253.0,0MQ,"Multi-Aero, Inc. d/b/a Air Choice One",13930.0,ORD,11288.0,DEC,2018,9
1,0,0,0,110,0MQ,21253.0,0MQ,"Multi-Aero, Inc. d/b/a Air Choice One",15016.0,STL,11288.0,DEC,2018,9
2,0,0,0,949,0WQ,21352.0,0WQ,Avjet Corporation,10279.0,AMA,10800.0,BUR,2018,9
3,0,0,0,725,0WQ,21352.0,0WQ,Avjet Corporation,10372.0,ASE,10800.0,BUR,2018,9
4,0,0,0,18,0WQ,21352.0,0WQ,Avjet Corporation,10800.0,BUR,12892.0,LAX,2018,9


In [210]:
stats_df['ORIGIN'].nunique()

1249

## Join and Pivot

Let's solve the problem of comparing the number of passengers flown by each major airline flying out the large airports in the US. Here are the steps we need to perform to be able to do that:

1. Identify all large airports in the US from airport codes dataframe.
2. Inner join the stats df with a df containing only the large airports
3. pivot on the origin airport code to get the stats for each airline in different columns.
4. Filter only large airlines.

In [5]:
codes_df['type'].unique()

array(['heliport', 'small_airport', 'seaplane_base', 'balloonport',
       'medium_airport', 'large_airport'], dtype=object)

In [10]:
large_airports_df = codes_df[(codes_df['type'] == 'seaplane_base') & (codes_df['iso_country'] == 'US')][['iso_region', 'name', 'municipality', 'local_code']]
large_airports_df.head(10)

,iso_region,name,municipality,local_code
36,US-NY,Cliche Cove Seaplane Base,Beekmantown,00NK
92,US-MD,Annapolis Seaplane Base,Annapolis,01MD
93,US-ME,Saint Peter's Seaplane Base,Portage Lake,01ME
95,US-MN,Barnes Seaplane Base,Cook,01MN
162,US-NH,Iroquois Landing Seaplane Base,Dummer,02NH
190,US-AK,Joe Clouds Seaplane Base,Kenai,03AK
211,US-ME,Lakeside Marina Seaplane Base,East Winthrop,03M
266,US-IN,Lake Gage Seaplane Base,Angola,04IN
314,US-ME,Van Buren Seaplane Base,Van Buren,05B
372,US-ME,Lucky Landing Marina and Seaplane Base,Bangor,06B


In [11]:
large_airport_stats_df = pd.merge(stats_df, large_airports_df, left_on='ORIGIN', right_on='local_code', how='inner')
large_airport_stats_df.head(10)

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,YEAR,MONTH,iso_region,name,municipality,local_code
0,0,20,0,84,2O,20443.0,2O,Island Air Service,10056.0,A43,12743.0,KMY,2018,9,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
1,0,131,31,46,2O,20443.0,2O,Island Air Service,10056.0,A43,12782.0,KPR,2018,9,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
2,0,296,30,93,2O,20443.0,2O,Island Air Service,10056.0,A43,10278.0,ALZ,2018,10,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
3,0,46,37,46,2O,20443.0,2O,Island Air Service,10056.0,A43,12782.0,KPR,2018,10,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
4,0,0,152,26,2O,20443.0,2O,Island Air Service,10056.0,A43,12785.0,KPY,2018,10,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
5,0,272,175,43,2O,20443.0,2O,Island Air Service,10056.0,A43,12854.0,KWP,2018,10,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
6,0,414,461,53,2O,20443.0,2O,Island Air Service,10056.0,A43,12869.0,KZB,2018,10,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
7,0,387,243,34,2O,20443.0,2O,Island Air Service,10056.0,A43,15446.0,UGI,2018,10,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
8,0,186,0,28,2O,20443.0,2O,Island Air Service,10056.0,A43,11513.0,EHR,2018,2,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43
9,0,165,18,46,2O,20443.0,2O,Island Air Service,10056.0,A43,12782.0,KPR,2018,2,US-AK,Taku Harbor Seaplane Base,Taku Harbor,A43


In [211]:
large_airport_stats_df['ORIGIN'].nunique()

157

In [215]:
large_airport_stats_df[(large_airport_stats_df['ORIGIN'] == 'ABQ') & (large_airport_stats_df['CARRIER_NAME'] == 'Air Transport International')]

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,DEST_AIRPORT_ID,DEST,YEAR,MONTH,iso_region,name,municipality,local_code


In [216]:
pivot_df = pd.pivot_table(large_airport_stats_df, 
                          index=['ORIGIN', 'iso_region', 'name', 'municipality'], 
                          columns=['CARRIER_NAME'], 
                          values='PASSENGERS', 
                          aggfunc=sum,
                          fill_value=0)
pivot_df.head(10)

,,,CARRIER_NAME,40-Mile Air,ABX Air Inc,"ADVANCED AIR, LLC",Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways,Air Transport International,Air Wisconsin Airlines Corp,Alaska Airlines Inc.,Alaska Central Express,Allegiant Air,Aloha Air Cargo,...,USA Jet Airlines Inc.,Ultimate JetCharters LLC dba Ultimate Air Shuttle,United Air Lines Inc.,United Parcel Service,Via Airlines d/b/a Charter Air Transport,Virgin America,Warbelow,Western Global,Wright Air Service,XTRA Airways
ORIGIN,iso_region,name,municipality,,,,,,,,,,,,,,,,,,,,,
ABQ,US-NM,Albuquerque International Sunport,Albuquerque,0,0,0,0,0,0,33682,0,28698,0,...,0,0,108961,0,0,0,0,0,0,15
ADW,US-MD,Andrews Air Force Base,Camp Springs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AFW,US-TX,Fort Worth Alliance Airport,Fort Worth,0,0,0,29,0,0,0,0,804,0,...,0,0,108,0,0,0,0,0,0,0
AGS,US-GA,Augusta Regional At Bush Field,Augusta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,389
AMA,US-TX,Rick Husband Amarillo International Airport,Amarillo,0,0,13,1,0,0,0,0,0,0,...,0,0,0,0,910,0,0,0,0,0
ANC,US-AK,Ted Stevens Anchorage International Airport,Anchorage,4,0,0,0,0,0,1707632,2569,0,0,...,0,0,135994,0,0,0,0,0,0,0
ATL,US-GA,Hartsfield Jackson Atlanta International Airport,Atlanta,0,0,6,70,0,0,101508,0,528,0,...,0,0,546920,0,0,0,0,0,0,391
AUS,US-TX,Austin Bergstrom International Airport,Austin,0,0,0,0,24,0,207375,0,139113,0,...,0,0,919824,0,11487,17456,0,0,0,0
AVL,US-NC,Asheville Regional Airport,Asheville,0,0,0,0,0,0,0,0,212253,0,...,0,0,23273,0,0,0,0,0,0,0


In [217]:
len(pivot_df.columns)

99

In [132]:
imp_airlines = pivot_df.columns[pivot_df.apply(lambda x: sum(x) > 10 * 1000000, axis=0)]
imp_airlines

Index(['Alaska Airlines Inc.', 'American Airlines Inc.',
       'Delta Air Lines Inc.', 'Endeavor Air Inc.', 'Envoy Air',
       'Frontier Airlines Inc.', 'JetBlue Airways', 'Mesa Airlines Inc.',
       'PSA Airlines Inc.', 'Republic Airline', 'SkyWest Airlines Inc.',
       'Southwest Airlines Co.', 'Spirit Air Lines', 'United Air Lines Inc.'],
      dtype='object', name='CARRIER_NAME')

In [133]:
airlines_compare_df = pivot_df[imp_airlines]
airlines_compare_df.head(10)

,,,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
ORIGIN,iso_region,name,municipality,,,,,,,,,,,,,,
ABQ,US-NM,Albuquerque International Sunport,Albuquerque,33682,307425,191373,0,7301,54361,42836,178666,0,25454,146675,1282912,0,108961
ADW,US-MD,Andrews Air Force Base,Camp Springs,0,0,0,0,0,0,0,0,0,0,0,0,0,0
AFW,US-TX,Fort Worth Alliance Airport,Fort Worth,0,0,0,0,0,0,0,0,0,0,0,0,0,108
AGS,US-GA,Augusta Regional At Bush Field,Augusta,0,0,97318,54472,970,0,0,5,86523,1032,32411,0,0,0
AMA,US-TX,Rick Husband Amarillo International Airport,Amarillo,0,291,0,0,55964,0,0,77298,0,0,541,164082,0,0
ANC,US-AK,Ted Stevens Anchorage International Airport,Anchorage,1707632,37168,317423,0,0,0,24814,0,0,0,0,0,0,135994
ATL,US-GA,Hartsfield Jackson Atlanta International Airport,Atlanta,101508,1270635,33239588,1472242,40113,530481,367303,154716,59166,293761,1058641,4890261,1119869,546920
AUS,US-TX,Austin Bergstrom International Airport,Austin,207375,1321041,974791,21174,4055,524069,282721,87094,2314,65473,174239,2729373,0,919824
AVL,US-NC,Asheville Regional Airport,Asheville,0,180,57854,38273,0,0,0,0,117489,0,87677,0,12316,23273


In [134]:
airlines_compare_df.index

MultiIndex(levels=[['ABQ', 'ADW', 'AFW', 'AGS', 'AMA', 'ANC', 'ATL', 'AUS', 'AVL', 'BAB', 'BAD', 'BDL', 'BFI', 'BGR', 'BHM', 'BIL', 'BLV', 'BMI', 'BNA', 'BOI', 'BOS', 'BUF', 'BWI', 'CAE', 'CHA', 'CHS', 'CID', 'CLE', 'CLT', 'CMH', 'CPR', 'CRP', 'CRW', 'CVG', 'DAB', 'DAL', 'DAY', 'DBQ', 'DCA', 'DEN', 'DFW', 'DLH', 'DOV', 'DSM', 'DTW', 'DYS', 'EDW', 'ERI', 'EWR', 'FAI', 'FFO', 'FLL', 'FSM', 'FTW', 'FWA', 'GEG', 'GPT', 'GRB', 'GSB', 'GSO', 'GSP', 'GUS', 'HIB', 'HNL', 'HOU', 'HSV', 'HTS', 'IAD', 'IAH', 'ICT', 'IND', 'JAN', 'JAX', 'JFK', 'JLN', 'LAS', 'LAX', 'LBB', 'LCK', 'LEX', 'LFI', 'LFT', 'LGA', 'LIT', 'MBS', 'MCI', 'MCO', 'MDW', 'MEM', 'MGE', 'MGM', 'MIA', 'MKE', 'MLI', 'MLU', 'MOB', 'MSN', 'MSP', 'MSY', 'MUO', 'OAK', 'OKC', 'OMA', 'ONT', 'ORD', 'ORF', 'PAM', 'PBI', 'PDX', 'PHF', 'PHL', 'PHX', 'PIA', 'PIT', 'PWM', 'RDU', 'RFD', 'RIC', 'RNO', 'ROA', 'ROC', 'RST', 'RSW', 'SAN', 'SAT', 'SAV', 'SBN', 'SDF', 'SEA', 'SFB', 'SFO', 'SGF', 'SJC', 'SLC', 'SMF', 'SNA', 'SPI', 'SPS', 'SRQ', 'SSC', 

In [223]:
final_airline_compare = airlines_compare_df.reorder_levels([1,3,0,2], axis=0)
final_airline_compare.head(5)

,,,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
iso_region,municipality,ORIGIN,name,,,,,,,,,,,,,,
US-NM,Albuquerque,ABQ,Albuquerque International Sunport,33682,307425,191373,0,7301,54361,42836,178666,0,25454,146675,1282912,0,108961
US-MD,Camp Springs,ADW,Andrews Air Force Base,0,0,0,0,0,0,0,0,0,0,0,0,0,0
US-TX,Fort Worth,AFW,Fort Worth Alliance Airport,0,0,0,0,0,0,0,0,0,0,0,0,0,108
US-GA,Augusta,AGS,Augusta Regional At Bush Field,0,0,97318,54472,970,0,0,5,86523,1032,32411,0,0,0
US-TX,Amarillo,AMA,Rick Husband Amarillo International Airport,0,291,0,0,55964,0,0,77298,0,0,541,164082,0,0


In [224]:
final_airline_compare.sort_index(level=[0, 1, 2], inplace=True)
final_airline_compare.head(5)

CARRIER_NAME                                                                   Alaska Airlines Inc.  \
iso_region municipality ORIGIN name                                                                   
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                  1707632   
           Fairbanks    FAI    Fairbanks International Airport                               389930   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport                     0   
           Huntsville   HSV    Huntsville International Carl T Jones Field                        0   
           Mobile       MOB    Mobile Regional Airport                                            0   

CARRIER_NAME                                                                   American Airlines Inc.  \
iso_region municipality ORIGIN name                                                                     
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                      37168   
           Fairbanks    FAI    Fairbanks International Airport                                      0   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport                     136   
           Huntsville   HSV    Huntsville International Carl T Jones Field                          0   
           Mobile       MOB    Mobile Regional Airport                                              0   

CARRIER_NAME                                                                   Delta Air Lines Inc.  \
iso_region municipality ORIGIN name                                                                   
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                   317423   
           Fairbanks    FAI    Fairbanks International Airport                                66385   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport                392466   
           Huntsville   HSV    Huntsville International Carl T Jones Field                   227371   
           Mobile       MOB    Mobile Regional Airport                                        38394   

CARRIER_NAME                                                                   Endeavor Air Inc.  \
iso_region municipality ORIGIN name                                                                
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                     0   
           Fairbanks    FAI    Fairbanks International Airport                                 0   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport              27377   
           Huntsville   HSV    Huntsville International Carl T Jones Field                 11674   
           Mobile       MOB    Mobile Regional Airport                                     17116   

CARRIER_NAME                                                                   Envoy Air  \
iso_region municipality ORIGIN name                                                        
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport             0   
           Fairbanks    FAI    Fairbanks International Airport                         0   
US-AL      Birmingham   BHM    Birmingham-Shuttlesworth International Airport      58441   
           Huntsville   HSV    Huntsville International Carl T Jones Field          8391   
           Mobile       MOB    Mobile Regional Airport                                56   

CARRIER_NAME                                                                   Frontier Airlines Inc.  \
iso_region municipality ORIGIN name                                                                     
US-AK      Anchorage    ANC    Ted Stevens Anchorage International Airport                          0   
           Fairbanks    FAI    Fairbanks International Airport                                      0   
US-AL      Birmingham   BHM    Birmingham-Shuttleswort

In [228]:
final_airline_compare.reset_index(level=3)

CARRIER_NAME                                                                      name  \
iso_region municipality      ORIGIN                                                      
US-AK      Anchorage         ANC           Ted Stevens Anchorage International Airport   
           Fairbanks         FAI                       Fairbanks International Airport   
US-AL      Birmingham        BHM        Birmingham-Shuttlesworth International Airport   
           Huntsville        HSV           Huntsville International Carl T Jones Field   
           Mobile            MOB                               Mobile Regional Airport   
           Montgomery        MGM          Montgomery Regional (Dannelly Field) Airport   
US-AR      Fort Smith        FSM                           Fort Smith Regional Airport   
           Little Rock       LIT     Bill & Hillary Clinton National Airport/Adams ...   
US-AZ      Phoenix           PHX              Phoenix Sky Harbor International Airport   
           Tucson            TUS                          Tucson International Airport   
US-CA      Edwards           EDW                                Edwards Air Force Base   
           Fairfield         SUU                                 Travis Air Force Base   
           Lompoc            VBG                             Vandenberg Air Force Base   
           Los Angeles       LAX                     Los Angeles International Airport   
           Marysville        BAB                                  Beale Air Force Base   
           Oakland           OAK            Metropolitan Oakland International Airport   
           Ontario           ONT                         Ontario International Airport   
           Sacramento        SMF                      Sacramento International Airport   
           San Diego         SAN                       San Diego International Airport   
           San Francisco     SFO                   San Francisco International Airport   
           San Jose          SJC       Norman Y. Mineta San Jose International Airport   
           Santa Ana         SNA              John Wayne Airport-Orange County Airport   
US-CO      Denver            DEN                          Denver International Airport   
US-CT      Hartford          BDL                         Bradley International Airport   
US-DC      Washington        DCA             Ronald Reagan Washington National Airport   
                             IAD               Washington Dulles International Airport   
US-DE      Dover             DOV                                  Dover Air Force Base   
US-FL      Daytona Beach     DAB                   Daytona Beach International Airport   
           Fort Lauderdale   FLL       Fort Lauderdale Hollywood International Airport   
           Fort Myers        RSW               Southwest Florida International Airport   
...                                                                                ...   
US-TN      Nashville         BNA                       Nashville International Airport   
US-TX      Abilene           DYS                                  Dyess Air Force Base   
           Amarillo          AMA           Rick Husband Amarillo International Airport   
           Austin            AUS                Austin Bergstrom International Airport   
           Corpus Christi    CRP                  Corpus Christi International Airport   
           Dallas            DAL                                     Dallas Love Field   
           Dallas-Fort Worth DFW               Dallas Fort Worth International Airport   
           Fort Worth        AFW                           Fort Worth Alliance Airport   
                             FTW              Fort Worth Meacham International Airport   
           Houston           HOU                               William P Hobby Airport   
                             IAH          George Bush Intercontinental Houston Airport   
           Lubbock           LBB        

In [227]:
final_airline_compare.loc[('US-NY', 'New York')]

,CARRIER_NAME,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
ORIGIN,name,,,,,,,,,,,,,,
JFK,John F Kennedy International Airport,555773,2017625,4275329,1263384,162502,0,5174316,0,15518,145377,86827,0,0,108
LGA,La Guardia Airport,0,2597812,3004050,1427111,452801,167096,623309,88859,63438,1678189,437635,1372406,654418,1115610


In [135]:
airlines_compare_df.columns

Index(['Alaska Airlines Inc.', 'American Airlines Inc.',
       'Delta Air Lines Inc.', 'Endeavor Air Inc.', 'Envoy Air',
       'Frontier Airlines Inc.', 'JetBlue Airways', 'Mesa Airlines Inc.',
       'PSA Airlines Inc.', 'Republic Airline', 'SkyWest Airlines Inc.',
       'Southwest Airlines Co.', 'Spirit Air Lines', 'United Air Lines Inc.'],
      dtype='object', name='CARRIER_NAME')

In [164]:
final_airline_compare.columns.name = None
final_airline_compare

ORIGIN  \
iso_region municipality               
US-AK      Anchorage            ANC   
           Fairbanks            FAI   
US-AL      Birmingham           BHM   
           Huntsville           HSV   
           Mobile               MOB   
           Montgomery           MGM   
US-AR      Fort Smith           FSM   
           Little Rock          LIT   
US-AZ      Phoenix              PHX   
           Tucson               TUS   
US-CA      Edwards              EDW   
           Fairfield            SUU   
           Lompoc               VBG   
           Los Angeles          LAX   
           Marysville           BAB   
           Oakland              OAK   
           Ontario              ONT   
           Sacramento           SMF   
           San Diego            SAN   
           San Francisco        SFO   
           San Jose             SJC   
           Santa Ana            SNA   
US-CO      Denver               DEN   
US-CT      Hartford             BDL   
US-DC      Washington           DCA   
           Washington           IAD   
US-DE      Dover                DOV   
US-FL      Daytona Beach        DAB   
           Fort Lauderdale      FLL   
           Fort Myers           RSW   
...                             ...   
US-TN      Nashville            BNA   
US-TX      Abilene              DYS   
           Amarillo             AMA   
           Austin               AUS   
           Corpus Christi       CRP   
           Dallas               DAL   
           Dallas-Fort Worth    DFW   
           Fort Worth           AFW   
           Fort Worth           FTW   
           Houston              HOU   
           Houston              IAH   
           Lubbock              LBB   
           San Antonio          SAT   
           Wichita Falls        SPS   
US-UT      Salt Lake City       SLC   
US-VA      Hampton              LFI   
           Newport News         PHF   
           Norfolk              ORF   
           Richmond             RIC   
           Roanoke              ROA   
US-WA      Seattle              BFI   
           Seattle              SEA   
           Spokane              GEG   
           Tacoma               TCM   
US-WI      Green Bay            GRB   
           Madison              MSN   
           Milwaukee            MKE   
US-WV      Charleston           CRW   
           Huntington           HTS   
US-WY      Casper               CPR   

                                                                           name  \
iso_region municipality                                                           
US-AK      Anchorage                Ted Stevens Anchorage International Airport   
           Fairbanks                            Fairbanks International Airport   
US-AL      Birmingham            Birmingham-Shuttlesworth International Airport   
           Huntsville               Huntsville International Carl T Jones Field   
           Mobile                                       Mobile Regional Airport   
           Montgomery              Montgomery Regional (Dannelly Field) Airport   
US-AR      Fort Smith                               Fort Smith Regional Airport   
           Little Rock        Bill & Hillary Clinton National Airport/Adams ...   
US-AZ      Phoenix                     Phoenix Sky Harbor International Airport   
           Tucson                                  Tucson International Airport   
US-CA      Edwards                                       Edwards Air Force Base   
           Fairfield                                      Travis Air Force Base   
           Lompoc                                     Vandenberg Air Force Base   
           Los Angeles                        Los Angeles International Airport   
           Marysville                                      Beale Air Force Base   
           Oakland                   Metropolitan Oakland International Airport   
           Ontario                                Ontario International Airpo

In [162]:
final_airline_compare.loc['US-CA']

,ORIGIN,name,Alaska Airlines Inc.,American Airlines Inc.,Delta Air Lines Inc.,Endeavor Air Inc.,Envoy Air,Frontier Airlines Inc.,JetBlue Airways,Mesa Airlines Inc.,PSA Airlines Inc.,Republic Airline,SkyWest Airlines Inc.,Southwest Airlines Co.,Spirit Air Lines,United Air Lines Inc.
municipality,,,,,,,,,,,,,,,,
Edwards,EDW,Edwards Air Force Base,0,0,0,0,19,0,0,0,0,0,0,0,0,0
Fairfield,SUU,Travis Air Force Base,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Lompoc,VBG,Vandenberg Air Force Base,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Los Angeles,LAX,Los Angeles International Airport,2416490,5825125,5223407,0,0,179168,891197,0,0,0,1644517,4834977,1305133,4602766
Marysville,BAB,Beale Air Force Base,0,0,0,0,22,0,0,0,0,0,0,0,0,0
Oakland,OAK,Metropolitan Oakland International Airport,352758,132511,146234,0,0,0,188995,24821,0,0,30611,4516438,404116,2493
Ontario,ONT,Ontario International Airport,153162,375494,30931,0,0,120847,16060,42672,0,0,149958,1356857,0,127096
Sacramento,SMF,Sacramento International Airport,274500,560065,355610,0,0,21862,163098,5249,0,0,312504,3187500,0,441881
San Diego,SAN,San Diego International Airport,1105456,1338206,1220542,0,98,274772,253409,0,0,0,561375,4524041,338731,1489952


In [138]:
pivot_df2 = pd.pivot_table(large_airport_stats_df, index=['ORIGIN', 'iso_region', 'name', 'municipality'], columns=['CARRIER_NAME'], values=['PASSENGERS', 'FREIGHT'], aggfunc=sum, fill_value=0)
pivot_df2.head(10)

FREIGHT  \
CARRIER_NAME                                                                    40-Mile Air   
ORIGIN iso_region name                                             municipality               
ABQ    US-NM      Albuquerque International Sunport                Albuquerque            0   
ADW    US-MD      Andrews Air Force Base                           Camp Springs           0   
AFW    US-TX      Fort Worth Alliance Airport                      Fort Worth             0   
AGS    US-GA      Augusta Regional At Bush Field                   Augusta                0   
AMA    US-TX      Rick Husband Amarillo International Airport      Amarillo               0   
ANC    US-AK      Ted Stevens Anchorage International Airport      Anchorage            100   
ATL    US-GA      Hartsfield Jackson Atlanta International Airport Atlanta                0   
AUS    US-TX      Austin Bergstrom International Airport           Austin                 0   
AVL    US-NC      Asheville Regional Airport                       Asheville              0   
BAB    US-CA      Beale Air Force Base                             Marysville             0   

                                                                                             \
CARRIER_NAME                                                                    ABX Air Inc   
ORIGIN iso_region name                                             municipality               
ABQ    US-NM      Albuquerque International Sunport                Albuquerque            0   
ADW    US-MD      Andrews Air Force Base                           Camp Springs           0   
AFW    US-TX      Fort Worth Alliance Airport                      Fort Worth             0   
AGS    US-GA      Augusta Regional At Bush Field                   Augusta                0   
AMA    US-TX      Rick Husband Amarillo International Airport      Amarillo               0   
ANC    US-AK      Ted Stevens Anchorage International Airport      Anchorage              0   
ATL    US-GA      Hartsfield Jackson Atlanta International Airport Atlanta         10142627   
AUS    US-TX      Austin Bergstrom International Airport           Austin             11625   
AVL    US-NC      Asheville Regional Airport                       Asheville              0   
BAB    US-CA      Beale Air Force Base                             Marysville             0   

                                                                                                   \
CARRIER_NAME                                                                    ADVANCED AIR, LLC   
ORIGIN iso_region name                                             municipality                     
ABQ    US-NM      Albuquerque International Sunport                Albuquerque                  0   
ADW    US-MD      Andrews Air Force Base                           Camp Springs                 0   
AFW    US-TX      Fort Worth Alliance Airport                      Fort Worth                   0   
AGS    US-GA      Augusta Regional At Bush Field                   Augusta                      0   
AMA    US-TX      Rick Husband Amarillo International Airport      Amarillo                     0   
ANC    US-AK      Ted Stevens Anchorage International Airport      Anchorage                    0   
ATL    US-GA      Hartsfield Jackson Atlanta International Airport Atlanta                      0   
AUS    US-TX      Austin Bergstrom International Airport           Austin                       0   
AVL    US-NC      Asheville Regional Airport                       Asheville                    0   
BAB    US-CA      Beale Air Force Base                             Marysville                   0   

                                                                                                                                         \
CARRIER_NAME                                                                    Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways   
ORIGIN iso_region na

In [139]:
pivot_df2.columns

MultiIndex(levels=[['FREIGHT', 'PASSENGERS'], ['40-Mile Air', 'ABX Air Inc', 'ADVANCED AIR, LLC', 'Aerodynamics Inc. d/b/a SkyValue d/b/a SkyValue Airways', 'Air Transport International', 'Air Wisconsin Airlines Corp', 'Alaska Airlines Inc.', 'Alaska Central Express', 'Allegiant Air', 'Aloha Air Cargo', 'American Airlines Inc.', 'Amerijet International', 'Ameristar Air Cargo', 'Asia Pacific', 'Atlas Air Inc.', 'Avjet Corporation', 'Bemidji Airlines', 'Bering Air Inc.', 'Boutique Air', 'CFM Inc d/b/a Contour Airlines d/b/a One Jet Shuttle', 'Cape Air', 'Caribbean Sun Airlines, Inc. d/b/a World Atlantic Airlines', 'Commutair Aka Champlain Enterprises, Inc.', 'Compass Airlines', 'Corvus Airlines, Inc d/b/a Era Aviation d/b/a Ravn Alaska', 'Delta Air Lines Inc.', 'Delux Public Charter LLC', 'Eastern Airlines f/k/a Dynamic Airways, LLC', 'Elite Airways LLC', 'Empire Airlines Inc.', 'Endeavor Air Inc.', 'Envoy Air', 'ExpressJet Airlines Inc.', 'Federal Express Corporation', 'Frontier Airline

In [140]:
pivot_df2['FREIGHT', 'United Parcel Service']

ORIGIN  iso_region  name                                                          municipality             
ABQ     US-NM       Albuquerque International Sunport                             Albuquerque                    39087941
ADW     US-MD       Andrews Air Force Base                                        Camp Springs                          0
AFW     US-TX       Fort Worth Alliance Airport                                   Fort Worth                            0
AGS     US-GA       Augusta Regional At Bush Field                                Augusta                               0
AMA     US-TX       Rick Husband Amarillo International Airport                   Amarillo                              0
ANC     US-AK       Ted Stevens Anchorage International Airport                   Anchorage                     523298938
ATL     US-GA       Hartsfield Jackson Atlanta International Airport              Atlanta                        47303207
AUS     US-TX       Austin Bergstrom I

## Group By

Let's solve a couple of problems to helps us understand the `DataFrame`'s `groupby()` method by better.

### Problem 1

Find the passenger count and freight load for each route among the large airports in the US. Find the busiest routes for passengers and freights in the US.

In [176]:
large_route_stats = pd.merge(large_airport_stats_df, large_airports_df, left_on='DEST', right_on='local_code')
large_route_stats.head(10)

,PASSENGERS,FREIGHT,MAIL,DISTANCE,UNIQUE_CARRIER,AIRLINE_ID,CARRIER,CARRIER_NAME,ORIGIN_AIRPORT_ID,ORIGIN,...,YEAR,MONTH,iso_region_x,name_x,municipality_x,local_code_x,iso_region_y,name_y,municipality_y,local_code_y
0,0,1309768,0,2846,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,...,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
1,0,1532654,0,2846,KD,21629.0,KD,Western Global,13930.0,ORD,...,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
2,0,1011342,0,2846,PO,20100.0,PO,Polar Air Cargo Airways,13930.0,ORD,...,2018,9,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
3,0,2687090,0,2846,5Y,20007.0,5Y,Atlas Air Inc.,13930.0,ORD,...,2018,1,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
4,0,1494827,0,2846,5Y,20007.0,5Y,Atlas Air Inc.,13930.0,ORD,...,2018,2,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
5,0,854986,0,2846,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,...,2018,2,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
6,0,1335407,0,2846,KD,21629.0,KD,Western Global,13930.0,ORD,...,2018,5,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
7,0,1074212,0,2846,5Y,20007.0,5Y,Atlas Air Inc.,13930.0,ORD,...,2018,7,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
8,0,2113243,0,2846,KAQ,20370.0,KAQ,Kalitta Air LLC,13930.0,ORD,...,2018,8,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC
9,0,1830741,0,2846,KD,21629.0,KD,Western Global,13930.0,ORD,...,2018,8,US-IL,Chicago O'Hare International Airport,Chicago,ORD,US-AK,Ted Stevens Anchorage International Airport,Anchorage,ANC


In [178]:
summary_stats = large_route_stats.groupby(['ORIGIN', 'municipality_x', 'DEST', 'municipality_y'])['PASSENGERS', 'FREIGHT'].sum()
summary_stats.head(10)

PASSENGERS  FREIGHT
ORIGIN municipality_x DEST municipality_y                     
ABQ    Albuquerque    ABQ  Albuquerque              0     1655
                      AFW  Fort Worth             227        0
                      AMA  Amarillo                22      171
                      ANC  Anchorage                0        0
                      ATL  Atlanta             132649    31513
                      AUS  Austin               45961     6360
                      BDL  Hartford                64        0
                      BHM  Birmingham              34        0
                      BNA  Nashville             1908     9669
                      BOI  Boise                  306       33

In [230]:
summary_stats.sort_values(by='FREIGHT', ascending=False)

PASSENGERS    FREIGHT
ORIGIN municipality_x    DEST municipality_y                          
ANC    Anchorage         SDF  Louisville                  0  455897457
                         CVG  Cincinnati                  0  378488076
LAX    Los Angeles       MEM  Memphis                 52293  249759069
ANC    Anchorage         ORD  Chicago                 76637  230537825
LAX    Los Angeles       HNL  Honolulu              1148693  170798479
ANC    Anchorage         LAX  Los Angeles             72506  155891164
EWR    Newark            MEM  Memphis                 41910  155157363
OAK    Oakland           MEM  Memphis                  3621  146433056
MEM    Memphis           LAX  Los Angeles             53382  144300307
                         EWR  Newark                  40988  143671754
ANC    Anchorage         MEM  Memphis                     0  130819244
HNL    Honolulu          LAX  Los Angeles           1154762  129977645
SDF    Louisville        ANC  Anchorage                   0  129888723
MEM    Memphis           OAK  Oakland                  3609  126178800
ONT    Ontario           SDF  Louisville                 50  121092455
MEM    Memphis           ANC  Anchorage                   7  120649056
ONT    Ontario           MEM  Memphis                     0  116171862
DFW    Dallas-Fort Worth SDF  Louisville             115124  112364542
MEM    Memphis           SEA  Seattle                     7  103335937
CVG    Cincinnati        ANC  Anchorage                   0  102418498
SDF    Louisville        EWR  Newark                  42016  101390569
MEM    Memphis           MIA  Miami                   25043  100872450
SDF    Louisville        ONT  Ontario                    17   98241056
MIA    Miami             MEM  Memphis                 23197   96666068
EWR    Newark            SDF  Louisville              43314   96167376
SDF    Louisville        PHL  Philadelphia            42614   94122490
LAX    Los Angeles       SFO  San Francisco         2000078   91402022
MEM    Memphis           DEN  Denver                 100905   91222137
SDF    Louisville        DFW  Dallas-Fort Worth      114352   89783078
ORD    Chicago           MEM  Memphis                146296   88922572
...                                                     ...        ...
PHL    Philadelphia      GEG  Spokane                    36          0
                         GPT  Gulfport                 1007          0
DTW    Detroit           RNO  Reno                      485          0
EWR    Newark            BOI  Boise                      43          0
                         CAE  Columbia                    0          0
                         GSP  Greenville              42707          0
                         CID  Cedar Rapids              119          0
                         GRB  Green Bay                 153          0
                         GEG  Spokane                    24          0
                         FWA  Fort Wayne               4665          0
                         EWR  Newark                      0          0
                         DSM  Des Moines              12433          0
                         DOV  Dover                       0          0
                         DAY  Dayton                  23731          0
                         CRP  Corpus Christi              0          0
PHF    Newport News      AGS  Augusta                     1          0
                         BHM  Birmingham                 11          0
                         DCA  Washington                 13          0
EWR    Newark            CHA  Chattanooga             16460          0
PHF    Newport News      FLL  Fort Lauderdale           148          0
                         IAD  Washington                 26          0
                         JFK  New York                    0          0
                         MGM  Montgomery                  0          0
                         ORF  Norfolk                     4          0
  

### Problem 2

List all routes which are serviced by at least 5 different airlines. A route is called serviced by an airline if more than 1,000 passengers fly that airline on that route each month in a year.

In [248]:
def num_carriers_service(group_df):    
    serviced_rows = group_df[group_df['PASSENGERS'] > 1000]
    unique_carriers = list(serviced_rows['CARRIER_NAME'].unique())
    return pd.Series([len(unique_carriers), str(unique_carriers)], index=['num_carriers', 'carriers'])

In [249]:
route_carriers = large_route_stats.groupby(['ORIGIN', 'municipality_x', 'DEST', 'municipality_y']).apply(num_carriers_service)
route_carriers.head(10)

num_carriers  \
ORIGIN municipality_x DEST municipality_y                 
ABQ    Albuquerque    ABQ  Albuquerque                0   
                      AFW  Fort Worth                 0   
                      AMA  Amarillo                   0   
                      ANC  Anchorage                  0   
                      ATL  Atlanta                    1   
                      AUS  Austin                     3   
                      BDL  Hartford                   0   
                      BHM  Birmingham                 0   
                      BNA  Nashville                  0   
                      BOI  Boise                      0   

                                                                                    carriers  
ORIGIN municipality_x DEST municipality_y                                                     
ABQ    Albuquerque    ABQ  Albuquerque                                                    []  
                      AFW  Fort Worth                                                     []  
                      AMA  Amarillo                                                       []  
                      ANC  Anchorage                                                      []  
                      ATL  Atlanta                                  ['Delta Air Lines Inc.']  
                      AUS  Austin          ['Allegiant Air', 'Southwest Airlines Co.', 'F...  
                      BDL  Hartford                                                       []  
                      BHM  Birmingham                                                     []  
                      BNA  Nashville                                                      []  
                      BOI  Boise                                                          []

In [250]:
route_carriers[route_carriers['num_carriers'] >= 5]

num_carriers  \
ORIGIN municipality_x DEST municipality_y                 
ABQ    Albuquerque    DEN  Denver                     6   
                      IAH  Houston                    5   
                      ORD  Chicago                    6   
ATL    Atlanta        DCA  Washington                 5   
                      DEN  Denver                     7   
                      IAH  Houston                    7   
                      LAX  Los Angeles                5   
                      LGA  New York                   5   
                      MCO  Orlando                    5   
                      ORD  Chicago                    8   
                      PHL  Philadelphia               5   
AUS    Austin         DEN  Denver                     6   
                      LAX  Los Angeles                6   
                      ORD  Chicago                    5   
                      SFO  San Francisco              5   
BDL    Hartford       ORD  Chicago                    8   
BHM    Birmingham     ORD  Chicago                    6   
BNA    Nashville      BOS  Boston                     5   
                      DEN  Denver                     6   
                      IAH  Houston                    5   
                      LGA  New York                   6   
                      MSP  Minneapolis                6   
                      ORD  Chicago                    8   
                      PHL  Philadelphia               5   
BOS    Boston         BNA  Nashville                  6   
                      JFK  New York                   8   
                      LAX  Los Angeles                6   
                      LGA  New York                   5   
                      PIT  Pittsburgh                 5   
                      RDU  Raleigh/Durham             6   
...                                                 ...   
SFO    San Francisco  JFK  New York                   5   
                      LAS  Las Vegas                  5   
                      LAX  Los Angeles                8   
                      PDX  Portland                   7   
                      PHX  Phoenix                    5   
                      SAN  San Diego                  5   
                      SEA  Seattle                    7   
                      SLC  Salt Lake City             5   
SJC    San Jose       DEN  Denver                     5   
                      LAX  Los Angeles                5   
                      SEA  Seattle                    5   
SLC    Salt Lake City DEN  Denver                     8   
                      IAH  Houston                    5   
                      LAS  Las Vegas                  5   
                      LAX  Los Angeles                5   
                      PHX  Phoenix                    6   
                      SFO  San Francisco              5   
SMF    Sacramento     LAX  Los Angeles                6   
                      SEA  Seattle                    5   
STL    St Louis       DEN  Denver                     7   
                      LGA  New York                   5   
                      MSP  Minneapolis                5   
                      ORD  Chicago                    9   
SYR    Syracuse       DTW  Detroit                    5   
                      ORD  Chicago                    5   
TPA    Tampa          MSP  Minneapolis                5   
                      RDU  Raleigh/Durham             5   
TUL    Tulsa          DEN  Denver                     7   
                      ORD  Chicago                    7   
TYS    Knoxville      ORD  Chicago                    6   

                                                                                    carriers  
ORIGIN municipality_x DEST municipality_y                                                     
ABQ    Albuquerque    DEN  Denver          ['Trans States Airlines', 'Frontier Airlines I...  
                      IAH  Houston      

### Exercise

Solve problem 2 above but with a new criteria - a flight is considered as servicing the route if it transports atleast 50,000 passengers in a year.

## Exercise set 1

1. For each elevation range 0-500 ft, 500-1000 ft, 1000-2000 ft, 2000-5000 ft and 5000-above ft, find the number of passengers flying out of airports which lie at that elevation.

2. Find the number of passengers and the amount of freight for each airport type in the US (heliport, small_airport, seaplane_base, balloonport, medium_airport, large_airport)

3. In every route, the number of passengers going forward in the route is not the same as the number of passengers going back. Let's say that people are migrating in one of the directions which has higher passengers. Find the routes with the largest amount of migration.

4. Same as 3, but for freight. In this case, it's likely even more the case that more freight is transferred in one direction than the opposite.

5. The busiest airports might change over the course of the year, due to changes in weather. Find the top 5 busiest airports in the US (measured by passenger travelling in to there) for each month of the year.

## Exercise set 2

For this set, we will use the date in the file `video_game_sales.csv`. This file contains the sales, ratings and metadata for most of the video games ever developed.

1. Find the top 10 publishers in the game industry by total global sales in the last 10 years.

2. For each genre, count how many games of that genre a publisher has made (there should be a column for each publisher). Limit this to top 10 publishers found in 1.

3. Same as in 2, but this time count how many highly rated games of each genre a publisher has made.
  - Highly rated is defined as follows: Across all games in the last 10 years, find the 80th percentile critic score and 80th percentile user score. All games whose critic score **and** the user score are higher than the 80th percentile scores are highly rated. If ratings are not provided, ignore that game.

4. For each publisher, how many unique developers have they worked with.

5. For each publisher, who is their most preferred developer? This is simply the developer who the publisher has worked with most often in the last 10 years.

6. For each publisher, who is their most successful developer by total global sales in the last 10 years?

7. For each publisher, find the total sales in each region, as well as global sales, for each game platform (there should be a column for each game platform). Sort the columns by total global sales of each platform.

8. For each value of age rating (`Rating` column), find the average critic score, average user score, and total sales in each region and globally.
